## Libreries

In [1]:
import pandas as pd
import pickle
import numpy as np

## Data

In [2]:
%%time
mf_features_train = pd.read_excel("../../feature-engineering/features/mf_features_train_task_C1.xlsx", index_col="id")
mf_features_val = pd.read_excel("../../feature-engineering/features/mf_features_val_task_C1.xlsx", index_col="id")
mf_features_test = pd.read_excel("../../feature-engineering/features/mf_features_test_task_C1.xlsx", index_col="id")

Wall time: 4min 11s


In [3]:
%%time
beto_features_train = pd.read_excel("../../feature-engineering/features/beto_pt_features_train_task_C1.xlsx", index_col="id")
beto_features_val = pd.read_excel("../../feature-engineering/features/beto_pt_features_val_task_C1.xlsx", index_col="id")
beto_features_test = pd.read_excel("../../feature-engineering/features/beto_pt_features_test_task_C1.xlsx", index_col="id")

Wall time: 3min 1s


In [4]:
A_train = pd.read_excel("../../data/train_task_C1.xlsx", index_col="id")
A_val = pd.read_excel("../../data/val_task_C1.xlsx", index_col="id")
A_test = pd.read_excel("../../data/test_task_C1.xlsx", index_col="id")

In [5]:
cols_beto = [f"beto<&>{x}" for x in range(768)]

In [6]:
beto_features_train.columns = cols_beto
beto_features_val.columns = cols_beto
beto_features_test.columns = cols_beto

In [7]:
X_train = pd.concat([mf_features_train, beto_features_train], axis=1)
X_val = pd.concat([mf_features_val, beto_features_val], axis=1)
X_test = pd.concat([mf_features_test, beto_features_test], axis=1)

In [8]:
y_train = A_train["label"]
X_train = X_train.loc[y_train.index]

y_test = A_test["label"]
X_test = X_test.loc[y_test.index]

y_val = A_val["label"]
X_val = X_val.loc[y_val.index]

In [9]:
!pip install xgboost

In [10]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [11]:
X_train.columns = range(X_train.shape[1])
X_val.columns = range(X_val.shape[1])
X_test.columns = range(X_test.shape[1])

In [12]:
%%time
xgb = XGBClassifier(max_depth=4, n_estimators=100, learning_rate=0.001, seed=2022)
xgb.fit(X_train, y_train)

Wall time: 35.5 s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.001, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=4, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=2022, ...)

In [13]:
y_pred = xgb.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96      2510
           1       0.86      0.60      0.71       388

    accuracy                           0.93      2898
   macro avg       0.90      0.79      0.84      2898
weighted avg       0.93      0.93      0.93      2898



In [14]:
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94       541
           1       0.90      0.58      0.71       136

    accuracy                           0.90       677
   macro avg       0.90      0.78      0.82       677
weighted avg       0.90      0.90      0.89       677



In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
parameters = {
    'max_depth': [6, 8, 10], 
    "n_estimators": [125, 100, 75],
    "learning_rate": [0.03, 0.1, 0.3],
}

In [17]:
clf = GridSearchCV(xgb, parameters, scoring="f1_macro", cv=5, verbose=1)

In [18]:
%%time
clf.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Wall time: 2h 24min 15s


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     callbacks=None, colsample_bylevel=1,
                                     colsample_bynode=1, colsample_bytree=1,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=0, gpu_id=-1,
                                     grow_policy='depthwise',
                                     importance_type=None,
                                     interaction_constraints='',
                                     learning_rate=0.001, max_b...6,
                                     max_cat_threshold=64, max_cat_to_onehot=4,
                                     max_delta_step=0, max_depth=4,
                                     max_leaves=0, min_child_weight=1,
                                     missing=nan, monotone_con

In [19]:
best_params = {}
for k in parameters:
    best_params[k] = vars(clf.best_estimator_)[k]
    print(f"{k}:", best_params[k])

max_depth: 10
n_estimators: 125
learning_rate: 0.3


In [20]:
y_pred = clf.predict(X_train)
report = classification_report(y_train, y_pred, output_dict=True)
train_report = pd.DataFrame(report)
train_report

,0,1,accuracy,macro avg,weighted avg
precision,0.999601,0.999349,0.999567,0.999475,0.999567
recall,0.999900,0.997403,0.999567,0.998651,0.999567
f1-score,0.999751,0.998375,0.999567,0.999063,0.999567
support,10019.000000,1540.000000,0.999567,11559.000000,11559.000000


In [21]:
y_pred = clf.predict(X_val)
report = classification_report(y_val, y_pred, output_dict=True)
val_report = pd.DataFrame(report)
val_report

,0,1,accuracy,macro avg,weighted avg
precision,0.976078,0.939655,0.971705,0.957867,0.971202
recall,0.991633,0.842784,0.971705,0.917208,0.971705
f1-score,0.983794,0.888587,0.971705,0.936191,0.971048
support,2510.000000,388.000000,0.971705,2898.000000,2898.000000


In [22]:
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred, output_dict=True)
test_report = pd.DataFrame(report)
test_report

,0,1,accuracy,macro avg,weighted avg
precision,0.932836,0.709220,0.886263,0.821028,0.887914
recall,0.924214,0.735294,0.886263,0.829754,0.886263
f1-score,0.928505,0.722022,0.886263,0.825263,0.887025
support,541.000000,136.000000,0.886263,677.000000,677.000000


## Save pickle

In [29]:
pickle.dump(clf.best_estimator_, open("results/clf_xgboost_mix_task_C1.pickle", "wb"))

In [24]:
pickle.dump(best_params, open("results/best_params_xgboost_mix_task_C1.pickle", "wb"))

In [25]:
pickle.dump(train_report, open("results/train_report_xgboost_mix_task_C1.pickle", "wb"))

In [26]:
pickle.dump(val_report, open("results/val_report_xgboost_mix_task_C1.pickle", "wb"))

In [27]:
pickle.dump(test_report, open("results/test_report_xgboost_mix_task_C1.pickle", "wb"))